In [26]:
import pandas as pd
import numpy as np
import ast
import re
from pandas.io.json import json_normalize
import nltk
from gensim.test.utils import common_texts
from gensim.models.word2vec import Word2Vec 
pd.set_option('display.max_columns', 200)
#pd.set_option('display.max_colwidth', -1)

In [27]:
df_hotencoded=pd.read_csv('yelpdataset_hotencoded.csv',low_memory=False)

In [28]:
df_hotencoded

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id,romantic,intimate,touristy,hipster,divey,classy,trendy,upscale,casual,American (New),Buffets,Restaurants,Nightlife,Lounges,Bars,American (Traditional),Breakfast & Brunch,Venues & Event Spaces,Event Planning & Services,Steakhouses,Gastropubs,Seafood,Burgers,Nightlife,Bars,Dim Sum,Japanese,Asian Fusion,Chinese,Breakfast & Brunch,African,Pubs,American (New),Barbeque,Restaurants,Southern,Lounges,Dance Clubs,Pizza,Italian,Salad,Local Flavor,Shopping,Arts & Entertainment,Food,Indian,Casinos,Hotels & Travel,Hotels,Sandwiches,Cafes,Event Planning & Services,Noodles,Ramen,Buffets,American (Traditional),Mexican,New Mexican Cuisine,Barbeque,Spanish,Tex-Mex,Caterers,Brazilian,Korean,Desserts,Specialty Food,Vegan,Cocktail Bars,Comfort Food,Tapas/Small Plates,Sushi Bars,Arts & Entertainment,Music Venues,Adult Entertainment,French,Active Life,Golf,Resorts,Mexican,Tacos,Food Delivery Services,Juice Bars & Smoothies,Delis,Performing Arts,Food,French,Soup,Wraps,Hotels,Fish & Chips,British,Casinos,Italian,Bakeries,Coffee & Tea,Car Dealers,Automotive,Wine Bars,Thai,Resorts,Meat Shops,Butcher,Steakhouses,Ethnic Food,Juice Bars & Smoothies,Social Clubs,Latin American,Vegetarian,Beer,Wine & Spirits,Vietnamese,Pizza,Sandwiches,Breweries,Cheese Shops,Specialty Food,Imported Food,Active Life,Amusement Parks,Cheesesteaks,Caterers,Beauty & Spas,Day Spas,Karaoke,Soul Food,Street Vendors,Hair Salons,Beauty & Spas,Professional Services,Employment Agencies,Hotels & Travel
0,3600 S Las Vegas Blvd,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...",ujHiaprwCQ5ewziu0Vi9rw,"American (New), Buffets, Restaurants",Las Vegas,"{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,36.113220,-115.176890,The Buffet at Bellagio,89109,4227,3.5,NV,0,2013-12-07 00:14:06,0,f0B9-r14-bLudyu5S7aLhw,1,We had dinner at the Bellagio Buffet last nigh...,1,SvMGr_Oih7ivtzmnCL-Tmg,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3600 S Las Vegas Blvd,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...",ujHiaprwCQ5ewziu0Vi9rw,"American (New), Buffets, Restaurants",Las Vegas,"{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,36.113220,-115.176890,The Buffet at Bellagio,89109,4227,3.5,NV,0,2015-12-05 03:28:19,0,A4G1-LTupScgaPmXF1OPMg,4,Around $45 for a dinner on a Friday night.\nSn...,0,W2WICriV5Zf03S4ETzEsiQ,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3600 S Las Vegas Blvd,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...",ujHiaprwCQ5ewziu0Vi9rw,"American (New), Buffets, Restaurants",Las Vegas,"{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,36.113220,-115.176890,The Buffet at Bellagio,89109,4227,3.5,NV,0,2017-02-09 04:48:06,0,aYSR8tO4kBqk0f6_ZYM2Zw,2,Went for Saturday Brunch. Wait was doable comp...,0,3BBiuiNLFuA-0Z4RE2UoQA,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3600 S Las Vegas Blvd,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...",ujHiaprwCQ5ewziu0Vi9rw,"American (New), Buffets, Restaurants",Las Vegas,"{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,36.113220,-115.176890,The Buffet at Bellagio,89109,4227,3.5,NV,1,2010-07-07 16:03:41,1,gCLbUG4PnhwhBKR_5jARjw,2,Having dined at many other Vegas buffets (the ...,4,7zphSXBrxDGChNlNkoGSng,0,0,0,

In [29]:
nltk.download('punkt') 



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/KennethMacBookPro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def clean_text(df_hotencoded_text):

    
    for pos in range(len(df_hotencoded_text)):
    
        #Get the comment
        val = df_hotencoded.iloc[pos]['text'];

        #Normalize tabs and remove newlines
        no_tabs = str(val).replace('\t', ' ').replace('\n', '');
        
        #Remove all characters except A-Z.
        alphas_only = re.sub("[^a-zA-Z]", " ", no_tabs);
        
        #Normalize spaces to 1
        multi_spaces = re.sub(" +", " ", alphas_only);
        
        #Strip trailing and leading spaces
        no_spaces = multi_spaces.strip();
        
        #Normalize all charachters to lowercase
        clean_text = no_spaces.lower();
        
        #Get sentences from the tokenizer, remove the dot in each.
        sentences = nltk.word_tokenize(clean_text)
        sentences = [re.sub("[\.]", "", sentence) for sentence in sentences];
    return sentences
    #return " ".join(sentences)

In [47]:
df_text_clean=pd.DataFrame(df_hotencoded[['business_id','user_id','text']])
df_text_clean.head()

,business_id,user_id,text
0,ujHiaprwCQ5ewziu0Vi9rw,SvMGr_Oih7ivtzmnCL-Tmg,We had dinner at the Bellagio Buffet last nigh...
1,ujHiaprwCQ5ewziu0Vi9rw,W2WICriV5Zf03S4ETzEsiQ,Around $45 for a dinner on a Friday night.\nSn...
2,ujHiaprwCQ5ewziu0Vi9rw,3BBiuiNLFuA-0Z4RE2UoQA,Went for Saturday Brunch. Wait was doable comp...
3,ujHiaprwCQ5ewziu0Vi9rw,7zphSXBrxDGChNlNkoGSng,Having dined at many other Vegas buffets (the ...
4,ujHiaprwCQ5ewziu0Vi9rw,XBkAAhQcrXQZ6fQsEp_KDQ,Still the best dinner buffet on the strip. Sta...


In [32]:
df_text_clean.shape

(254487, 3)

In [34]:
import time
import multiprocessing as mp

SetMultiprocessCore=10
num_rows=len(df_text_clean)
#num_rows=100
#NumParts=10
results=[]

#SplitRows=np.array_split(np.array(range(num_rows)),NumParts)

In [35]:
pool = mp.Pool(mp.cpu_count())

def multiprocess_obj(num_rows):
    if __name__ == '__main__':
        start=time.time()
        for i in range(num_rows):
            pool.apply_async(process_data, args=(i,), callback=collect_result)
        pool.close()
        pool.join()
        end = time.time()
        print ('Total Time Taken:', end-start)

In [36]:
from IPython.display import clear_output
def collect_result(data_tuple):
    global results
    clear_output(wait=True)
    print('Current Progress :',len(results),'of ',num_rows)
    return results.append(data_tuple)
    

In [37]:
from multiprocessing import Lock
lock=Lock()
def process_data(i):
    #print('i',i)
    lock.acquire()
    try:
        sentenceClean=clean_text(df_text_clean.loc[i,'text'])
        #df_text_clean.loc[i,'textclean']=sentenceClean
        data_tuple=(i,sentenceClean)
    finally:
        lock.release()
    return data_tuple

In [42]:
multiprocess_obj(num_rows)

Current Progress : 254486 of  254487
Total Time Taken: 116274.0080549717


In [43]:
results

[(0,
  ['very',
   'disappointed',
   'the',
   'quality',
   'of',
   'the',
   'food',
   'is',
   'not',
   'the',
   'same',
   'when',
   'we',
   'were',
   'her',
   'last',
   'prime',
   'rib',
   'dry',
   'soggy',
   'shrimp',
   'bad',
   'instant',
   'mash',
   'potatoes',
   'next',
   'time',
   'we',
   'will',
   'go',
   'to',
   'wynn']),
 (1,
  ['i',
   'really',
   'liked',
   'the',
   'quality',
   'of',
   'food',
   'and',
   'it',
   'is',
   'weird',
   'that',
   'i',
   'liked',
   'the',
   'coke',
   'they',
   'provide',
   'there',
   'were',
   'so',
   'many',
   'choices',
   'of',
   'food',
   'and',
   'i',
   'wished',
   'i',
   'had',
   'bigger',
   'stomach',
   'so',
   'i',
   'can',
   'eat',
   'more',
   'the',
   'desserts',
   'were',
   'absolutely',
   'delicious',
   'and',
   'not',
   'too',
   'sweet']),
 (2,
  ['the',
   'wait',
   'was',
   'not',
   'too',
   'bad',
   'although',
   'we',
   'went',
   'hour',
   'before',
 

In [44]:
df_results=pd.DataFrame(results, columns=['index','cleanText'])
df_results

,index,cleanText
0,0,"[very, disappointed, the, quality, of, the, fo..."
1,1,"[i, really, liked, the, quality, of, food, and..."
2,2,"[the, wait, was, not, too, bad, although, we, ..."
3,3,"[excellent, selection, of, seafood, meats, bre..."
4,4,"[usually, pretty, good, service, but, not, so,..."
5,5,"[i, have, eaten, at, the, best, buffets, in, v..."
6,6,"[the, use, of, the, word, gourmet, is, false, ..."
7,7,"[quality, control, at, its, poorest, i, spit, ..."
8,8,"[definitely, tops, my, list, as, the, best, bu..."
9,9,"[since, i, m, not, a, frequenter, of, buffets,..."


In [48]:
df_text_clean.head()

,business_id,user_id,text
0,ujHiaprwCQ5ewziu0Vi9rw,SvMGr_Oih7ivtzmnCL-Tmg,We had dinner at the Bellagio Buffet last nigh...
1,ujHiaprwCQ5ewziu0Vi9rw,W2WICriV5Zf03S4ETzEsiQ,Around $45 for a dinner on a Friday night.\nSn...
2,ujHiaprwCQ5ewziu0Vi9rw,3BBiuiNLFuA-0Z4RE2UoQA,Went for Saturday Brunch. Wait was doable comp...
3,ujHiaprwCQ5ewziu0Vi9rw,7zphSXBrxDGChNlNkoGSng,Having dined at many other Vegas buffets (the ...
4,ujHiaprwCQ5ewziu0Vi9rw,XBkAAhQcrXQZ6fQsEp_KDQ,Still the best dinner buffet on the strip. Sta...


In [49]:
df_text_clean=df_text_clean.merge(df_results,left_index=True, right_on='index')
del df_text_clean['index']
df_text_clean

,business_id,user_id,text,cleanText
0,ujHiaprwCQ5ewziu0Vi9rw,SvMGr_Oih7ivtzmnCL-Tmg,We had dinner at the Bellagio Buffet last nigh...,"[very, disappointed, the, quality, of, the, fo..."
1,ujHiaprwCQ5ewziu0Vi9rw,W2WICriV5Zf03S4ETzEsiQ,Around $45 for a dinner on a Friday night.\nSn...,"[i, really, liked, the, quality, of, food, and..."
2,ujHiaprwCQ5ewziu0Vi9rw,3BBiuiNLFuA-0Z4RE2UoQA,Went for Saturday Brunch. Wait was doable comp...,"[the, wait, was, not, too, bad, although, we, ..."
3,ujHiaprwCQ5ewziu0Vi9rw,7zphSXBrxDGChNlNkoGSng,Having dined at many other Vegas buffets (the ...,"[excellent, selection, of, seafood, meats, bre..."
4,ujHiaprwCQ5ewziu0Vi9rw,XBkAAhQcrXQZ6fQsEp_KDQ,Still the best dinner buffet on the strip. Sta...,"[usually, pretty, good, service, but, not, so,..."
5,ujHiaprwCQ5ewziu0Vi9rw,j1MjZ7f1DkYnsGNZ9gTK5w,I remember coming to the Buffet at the Bellagi...,"[i, have, eaten, at, the, best, buffets, in, v..."
6,ujHiaprwCQ5ewziu0Vi9rw,WJYs12ng86_q70n4CkdkTA,I had lunch at the buffet and it was by far th...,"[the, use, of, the, word, gourmet, is, false, ..."
7,ujHiaprwCQ5ewziu0Vi9rw,DeVGAiOf2mHVUDfxvuhVlQ,This place gives buffets a good name. The food...,"[quality, control, at, its, poorest, i, spit, ..."
8,ujHiaprwCQ5ewziu0Vi9rw,MuhUC0v3qBBTj6D2DO0H_w,Saturday night buffet around 9PM.\n\nAt 9PM th...,"[definitely, tops, my, list, as, the, best, bu..."
9,ujHiaprwCQ5ewziu0Vi9rw,cF3f4JQJ_N2IcEfTAM1-dA,I was expecting a lot more for what they charg...,"[since, i, m, not, a, frequenter, of, buffets,..."


In [50]:
df_text_clean.to_csv('yelpdataset_textclean.csv',index=False)